In [1]:
# -*- coding: utf-8 -*-
import sys; print('Python %s on %s' % (sys.version, sys.platform))
import os
import time
import json
from glob import glob, iglob
from tqdm import tqdm
import matplotlib.pyplot as plt

import numpy as np; print('numpy', np.__version__)
import pandas as pd; print('pandas', pd.__version__)
import cv2; print('opencv2', cv2.__version__)
import sklearn; print('sklearn', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import tensorflow.keras as keras; print('keras', keras.__version__)

import settings
import helper

Python 3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]] on linux
numpy 1.16.4
pandas 0.24.2
opencv2 4.1.0
sklearn 0.21.2
tensorflow 1.14.0
keras 2.2.4-tf


# 1. Load Meta File and Annotation

In [2]:
df_meta_train = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TRAIN_META_FILE, index_col=['seriesuid'])
df_meta_train.index = df_meta_train.index.astype('str')
df_meta_test = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TEST_META_FILE, index_col=['seriesuid'])
df_meta_test.index = df_meta_test.index.astype('str')

print('meta_train:', df_meta_train.shape, '\n', 'meta_test:', df_meta_test.shape)

meta_train: (1470, 11) 
 meta_test: (222, 11)


In [3]:
df_annotation = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_ANNOTATION_FILE, index_col=['seriesuid'])
df_annotation.index = df_annotation.index.astype('str')
df_annotation['label'] = df_annotation['label'].astype('int')
print('annotation:', df_annotation.shape, 'distinct lung:', len(set(df_annotation.index)))

annotation: (12218, 16) distinct lung: 1436


# 2. Load Center Coords by Candidates

In [4]:
lungs = list(set(df_meta_test.index))
print('distinct lungs in testset:', len(lungs))

distinct lungs in testset: 222


In [5]:
df_tasks = pd.read_csv(filepath_or_buffer=settings.SUBMISSION_DIR + 'candidates/tasks_lung.csv', 
                            index_col=['seriesuid'])
df_tasks.index = df_tasks.index.astype('str')

print('test tasks:', df_tasks.shape, 'distinct lung:', len(set(df_tasks.index)))

test tasks: (13024, 6) distinct lung: 222


In [6]:
# def nms(records, thresh):
#     x1 = records[:, 'vcoordX'] - records[:, 'diameterX'] / 2
#     y1 = records[:, 'vcoordY'] - records[:, 'diameterY'] / 2
#     x2 = records[:, 'vcoordX'] + records[:, 'diameterX'] / 2
#     y2 = records[:, 'vcoordY'] + records[:, 'diameterY'] / 2
#     scores = record[:, 'probability']

#     areas = (x2 - x1 + 1) * (y2 - y1 + 1)
#     order = scores.argsort()[::-1]

#     keep = []
#     while order.size > 0:
#         i = order[0]
#         keep.append(i)
#         xx1 = np.maximum(x1[i], x1[order[1:]])
#         yy1 = np.maximum(y1[i], y1[order[1:]])
#         xx2 = np.minimum(x2[i], x2[order[1:]])
#         yy2 = np.minimum(y2[i], y2[order[1:]])

#         w = np.maximum(0.0, xx2 - xx1 + 1)
#         h = np.maximum(0.0, yy2 - yy1 + 1)
#         inter = w * h
#         ovr = inter / (areas[i] + areas[order[1:]] - inter)

#         inds = np.where(ovr <= thresh)[0]
#         order = order[inds + 1]

#     return keep

# list_tasks_nms = []
# for uid in set(df_test_tasks.index):
#     tasks = df_test_tasks.loc[[uid]] # DataFrame
#     list_tasks_nms.extend(nms(tasks, 0.8))

# 3. Predict Results by Task_test.csv

In [7]:
df_tasks_nms = df_tasks

# df_tasks_nms = pd.DataFrame(list_tasks_nms, columns=['seriesuid','vcoordX','vcoordY','vcoordZ'])
# df_tasks_nms.index = df_tasks_nms.index.astype('str')

print('test tasks:', df_tasks_nms.shape, 'distinct lung:', len(set(df_tasks_nms.index)))

test tasks: (13024, 6) distinct lung: 222


In [8]:
df_tasks_nms.head(10)

,vcoordX,vcoordY,vcoordZ,diametersX,diametersY,diametersZ
seriesuid,,,,,,
678189,255,101,23,15,15,11
678189,224,104,78,6,6,3
678189,197,156,57,12,10,5
678189,75,105,104,7,6,8
678189,95,91,101,5,5,4
678189,214,159,95,7,8,8
678189,255,157,95,9,8,6
678189,220,185,94,8,10,14
678189,95,134,127,7,8,6


In [9]:
import tensorflow.keras.backend as K
# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()

# gpu allow_growth
config.gpu_options.allow_growth = True
keras.backend.set_session(tf.Session(config=config))

In [10]:
from tensorflow.keras.models import load_model
output_dir = './output/densenet3d/1564138475/' # labels in center coord
model = load_model(output_dir + "model-best.hd5")

W0729 13:06:28.187856 139795974772544 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0729 13:06:28.188754 139795974772544 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0729 13:06:28.195406 139795974772544 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprec

In [11]:
labels = [0, 1, 5]
batch_size = 32
threshold_probability = 0.7
wtype = 'lung'
plot = False

In [12]:
def plot_cube(cube, rows=8, cols=8):
    num_z, h, w = cube.shape # z,y,x
    assert rows * cols == num_z
    
    img = np.zeros((rows * h, cols * w), dtype=np.uint8)

    for row in range(rows):
        for col in range(cols):
            y = row * h
            x = col * w
            img[y:(y+h), x:(x+w)] = cube[row * cols + col,:,:]
    
    fig, axs = plt.subplots(1, 1, figsize=(32, 32), sharex=True, sharey=True)
    axs.imshow(img, cmap='gray')
    plt.show()
    

In [13]:
list_results = []
print(wtype, 'window')
count_lung = 0
for uid in set(df_tasks_nms.index):
    tasks = df_tasks_nms.loc[[uid]] # DataFrame
    meta = df_meta_test.loc[uid] # slice
    
    print('lung:', uid, 'tasks:', len(tasks))
#     print(meta)
                 

    lung_l, mask_l = helper.load_lung_array(uid, int(meta.width), int(meta.height), int(meta.slice), 
                                            mode='test', wtype=wtype)

    lung = (lung_l*(mask_l>0)).astype('float') / 255

    # convert coordX to vcoordX
#     tasks['vcoordX'] = tasks['coordX'] - meta.originX
#     tasks['vcoordY'] = tasks['coordY'] - meta.originY
#     tasks['vcoordZ'] = tasks['coordZ'] - meta.originZ

    count = 0
    for idx, item in tasks.iterrows():
        if count % (batch_size*100) == 0:
            X_item = []
            X_test = []
            
        flag_range = True
        if (item.vcoordX < 40) or (item.vcoordX > 360):
            flag_range = False
        if (item.vcoordY < 40) or (item.vcoordY > 300):
            flag_range = False
        if item.vcoordZ > 350:
            flag_range = False          
            
        # patch without mask
        cube = helper.get_cube_from_lung_array(lung, item.vcoordX, item.vcoordY, item.vcoordZ, 
                                               block_size=32)

        if np.sum(cube) > 0 and flag_range:
            X_item.append(item)
            X_test.append(cube)
        # else:
            # print('nothing in coord:', '\n', item)

        if (count % (batch_size*100) == (batch_size*100 - 1) or count == len(tasks) - 1) and len(X_test) > 0:
            assert len(X_item) == len(X_test) or count == len(tasks) - 1

            # Make predictions
            X_test = np.expand_dims(np.asarray(X_test), axis=-1)
            predictions_test = model.predict(X_test, batch_size=batch_size*5, verbose=1)

            for i in range(len(X_item)):
                itm = X_item[i]
                if np.argmax(predictions_test[i]) > 0:
                    category = int(np.argmax(predictions_test[i]))
                    probability = float(np.max(predictions_test[i]))

                    result = {}
                    result['seriesuid'] = uid
                    result['coordX'] = itm.vcoordX + meta.originX
                    result['coordY'] = itm.vcoordY + meta.originY
                    result['coordZ'] = itm.vcoordZ + meta.originZ
                    result['class'] = helper.get_origin_class_by_predicted_value(category, labels)
                    result['probability'] = probability
                    result['wtype'] = wtype

                    if probability > threshold_probability:
                        print(result)
                        if plot:
                            plot_cube((cube*255).astype(np.uint8), rows=4, cols=8)
                        list_results.append(result)

        count += 1
   
    # save the csv by per lung
    df_results = pd.DataFrame(list_results, columns=['seriesuid','coordX','coordY','coordZ','class','probability','wtype'])
    df_results = df_results.set_index('seriesuid')
    df_results.index = df_results.index.astype('str')

    count_lung += 1
    print('total:', df_results.shape, 'lung:', len(set(df_results.index)), '/', count_lung)

    df_results.to_csv(settings.SUBMISSION_DIR + 'candidates' + '/submission_' + wtype +'.csv', encoding='utf-8')

    print('-'*100)
    

lung window
lung: 664087 tasks: 60
58/58 [==============================] - 4s 70ms/sample
{'seriesuid': '664087', 'coordX': 80.50999999999999, 'coordY': -82.65799999999999, 'coordZ': -373.079, 'class': 1, 'probability': 0.9912202954292297, 'wtype': 'lung'}
{'seriesuid': '664087', 'coordX': 60.50999999999999, 'coordY': -10.657999999999987, 'coordZ': -244.079, 'class': 5, 'probability': 0.7035123109817505, 'wtype': 'lung'}
{'seriesuid': '664087', 'coordX': 64.50999999999999, 'coordY': -39.65799999999999, 'coordZ': -372.079, 'class': 5, 'probability': 0.9930891394615173, 'wtype': 'lung'}
{'seriesuid': '664087', 'coordX': -38.49000000000001, 'coordY': -110.65799999999999, 'coordZ': -340.079, 'class': 5, 'probability': 0.9793789386749268, 'wtype': 'lung'}
{'seriesuid': '664087', 'coordX': -41.49000000000001, 'coordY': -78.65799999999999, 'coordZ': -344.079, 'class': 5, 'probability': 0.9703980684280396, 'wtype': 'lung'}
{'seriesuid': '664087', 'coordX': 1.509999999999991, 'coordY': -19.657

62/62 [==============================] - 1s 13ms/sample
{'seriesuid': '688475', 'coordX': 56.34200000000001, 'coordY': 22.342000000000013, 'coordZ': -32.7513, 'class': 5, 'probability': 0.9948362112045288, 'wtype': 'lung'}
{'seriesuid': '688475', 'coordX': 58.34200000000001, 'coordY': 79.34200000000001, 'coordZ': -39.7513, 'class': 1, 'probability': 0.9410895705223083, 'wtype': 'lung'}
{'seriesuid': '688475', 'coordX': -83.65799999999999, 'coordY': 56.34200000000001, 'coordZ': -8.7513, 'class': 5, 'probability': 0.9886736273765564, 'wtype': 'lung'}
{'seriesuid': '688475', 'coordX': 10.342000000000013, 'coordY': 41.34200000000001, 'coordZ': 1.2486999999999995, 'class': 1, 'probability': 0.9975305199623108, 'wtype': 'lung'}
{'seriesuid': '688475', 'coordX': 24.342000000000013, 'coordY': 26.342000000000013, 'coordZ': 37.2487, 'class': 5, 'probability': 0.8753111958503723, 'wtype': 'lung'}
{'seriesuid': '688475', 'coordX': 91.34200000000001, 'coordY': -14.657999999999987, 'coordZ': 66.2487

39/39 [==============================] - 0s 12ms/sample
{'seriesuid': '688239', 'coordX': -24.105999999999995, 'coordY': -218.106, 'coordZ': -516.7, 'class': 1, 'probability': 0.7318092584609985, 'wtype': 'lung'}
{'seriesuid': '688239', 'coordX': -67.106, 'coordY': -144.106, 'coordZ': -515.7, 'class': 1, 'probability': 0.9999401569366455, 'wtype': 'lung'}
{'seriesuid': '688239', 'coordX': -56.105999999999995, 'coordY': -111.106, 'coordZ': -521.7, 'class': 1, 'probability': 0.8654060959815979, 'wtype': 'lung'}
{'seriesuid': '688239', 'coordX': -48.105999999999995, 'coordY': -150.106, 'coordZ': -480.70000000000005, 'class': 5, 'probability': 0.8055230975151062, 'wtype': 'lung'}
{'seriesuid': '688239', 'coordX': 60.894000000000005, 'coordY': -151.106, 'coordZ': -577.7, 'class': 5, 'probability': 0.9632255434989929, 'wtype': 'lung'}
{'seriesuid': '688239', 'coordX': -22.105999999999995, 'coordY': -231.106, 'coordZ': -552.7, 'class': 5, 'probability': 0.9587494730949402, 'wtype': 'lung'}
{'

40/40 [==============================] - 0s 9ms/sample
{'seriesuid': '672337', 'coordX': 48.34200000000001, 'coordY': 12.342000000000013, 'coordZ': -258.034, 'class': 5, 'probability': 0.8127440214157104, 'wtype': 'lung'}
{'seriesuid': '672337', 'coordX': -35.65799999999999, 'coordY': 60.34200000000001, 'coordZ': -253.034, 'class': 1, 'probability': 0.9940874576568604, 'wtype': 'lung'}
{'seriesuid': '672337', 'coordX': -34.65799999999999, 'coordY': 60.34200000000001, 'coordZ': -254.034, 'class': 1, 'probability': 0.9939960241317749, 'wtype': 'lung'}
{'seriesuid': '672337', 'coordX': -15.657999999999987, 'coordY': 17.342000000000013, 'coordZ': -181.034, 'class': 1, 'probability': 0.9997162222862244, 'wtype': 'lung'}
{'seriesuid': '672337', 'coordX': -78.65799999999999, 'coordY': 19.342000000000013, 'coordZ': -162.034, 'class': 1, 'probability': 0.9999741315841675, 'wtype': 'lung'}
{'seriesuid': '672337', 'coordX': -65.65799999999999, 'coordY': -46.65799999999999, 'coordZ': -251.034, 'cl

32/32 [==============================] - 0s 14ms/sample
{'seriesuid': '682302', 'coordX': -38.67400000000001, 'coordY': -217.67400000000004, 'coordZ': -582.1, 'class': 5, 'probability': 0.9867153763771057, 'wtype': 'lung'}
{'seriesuid': '682302', 'coordX': 86.326, 'coordY': -75.67400000000004, 'coordZ': -558.1, 'class': 1, 'probability': 0.9532710313796997, 'wtype': 'lung'}
{'seriesuid': '682302', 'coordX': 86.326, 'coordY': -80.67400000000004, 'coordZ': -552.1, 'class': 1, 'probability': 0.8299006819725037, 'wtype': 'lung'}
{'seriesuid': '682302', 'coordX': -49.67400000000001, 'coordY': -185.67400000000004, 'coordZ': -521.1, 'class': 5, 'probability': 0.7494505047798157, 'wtype': 'lung'}
{'seriesuid': '682302', 'coordX': 84.326, 'coordY': -112.67400000000004, 'coordZ': -513.1, 'class': 1, 'probability': 0.9998944997787476, 'wtype': 'lung'}
{'seriesuid': '682302', 'coordX': -49.67400000000001, 'coordY': -147.67400000000004, 'coordZ': -483.1, 'class': 5, 'probability': 0.988508522510528

...

67/67 [==============================] - 0s 3ms/sample
{'seriesuid': '341658', 'coordX': 130.49, 'coordY': 899.551, 'coordZ': 651.24, 'class': 5, 'probability': 0.9666556715965271, 'wtype': 'lung'}
{'seriesuid': '341658', 'coordX': -15.509999999999991, 'coordY': 948.551, 'coordZ': 715.24, 'class': 1, 'probability': 0.9844648241996765, 'wtype': 'lung'}
{'seriesuid': '341658', 'coordX': 67.49000000000001, 'coordY': 946.551, 'coordZ': 738.24, 'class': 1, 'probability': 0.8738929033279419, 'wtype': 'lung'}
{'seriesuid': '341658', 'coordX': 95.49000000000001, 'coordY': 946.551, 'coordZ': 745.24, 'class': 5, 'probability': 0.8676683306694031, 'wtype': 'lung'}
{'seriesuid': '341658', 'coordX': 106.49000000000001, 'coordY': 936.551, 'coordZ': 750.24, 'class': 5, 'probability': 0.9993637204170227, 'wtype': 'lung'}
{'seriesuid': '341658', 'coordX': -62.50999999999999, 'coordY': 910.551, 'coordZ': 776.24, 'class': 5, 'probability': 0.9510999321937561, 'wtype': 'lung'}
{'seriesuid': '341658', 'coo

54/54 [==============================] - 0s 6ms/sample
{'seriesuid': '684803', 'coordX': -113.12200000000001, 'coordY': -57.65799999999999, 'coordZ': -11.212999999999994, 'class': 1, 'probability': 0.989065408706665, 'wtype': 'lung'}
{'seriesuid': '684803', 'coordX': -75.12200000000001, 'coordY': 52.34200000000001, 'coordZ': 9.787000000000006, 'class': 1, 'probability': 0.9987936019897461, 'wtype': 'lung'}
{'seriesuid': '684803', 'coordX': 53.877999999999986, 'coordY': 49.34200000000001, 'coordZ': 36.787000000000006, 'class': 1, 'probability': 0.9996428489685059, 'wtype': 'lung'}
{'seriesuid': '684803', 'coordX': -3.122000000000014, 'coordY': 11.342000000000013, 'coordZ': 70.787, 'class': 1, 'probability': 0.8620436787605286, 'wtype': 'lung'}
{'seriesuid': '684803', 'coordX': -69.12200000000001, 'coordY': -42.65799999999999, 'coordZ': 106.787, 'class': 1, 'probability': 0.9997027516365051, 'wtype': 'lung'}
{'seriesuid': '684803', 'coordX': -13.122000000000014, 'coordY': 17.342000000000

17/17 [==============================] - 0s 5ms/sample
{'seriesuid': '672305', 'coordX': -77.668, 'coordY': -218.668, 'coordZ': -662.7, 'class': 5, 'probability': 0.9528684616088867, 'wtype': 'lung'}
{'seriesuid': '672305', 'coordX': -71.668, 'coordY': -183.668, 'coordZ': -604.7, 'class': 1, 'probability': 0.9914306998252869, 'wtype': 'lung'}
{'seriesuid': '672305', 'coordX': 41.331999999999994, 'coordY': -144.668, 'coordZ': -688.7, 'class': 5, 'probability': 0.9857591390609741, 'wtype': 'lung'}
{'seriesuid': '672305', 'coordX': 50.331999999999994, 'coordY': -143.668, 'coordZ': -688.7, 'class': 5, 'probability': 0.9895524382591248, 'wtype': 'lung'}
{'seriesuid': '672305', 'coordX': -68.668, 'coordY': -218.668, 'coordZ': -658.7, 'class': 5, 'probability': 0.9870246648788452, 'wtype': 'lung'}
{'seriesuid': '672305', 'coordX': 28.331999999999994, 'coordY': -135.668, 'coordZ': -667.7, 'class': 5, 'probability': 0.9968042373657227, 'wtype': 'lung'}
{'seriesuid': '672305', 'coordX': -39.6680

59/59 [==============================] - 1s 9ms/sample
{'seriesuid': '644447', 'coordX': 124.65400000000002, 'coordY': -72.60900000000001, 'coordZ': -433.139, 'class': 1, 'probability': 0.8683826923370361, 'wtype': 'lung'}
{'seriesuid': '644447', 'coordX': 88.65400000000002, 'coordY': -3.609000000000009, 'coordZ': -414.139, 'class': 5, 'probability': 0.7912085056304932, 'wtype': 'lung'}
{'seriesuid': '644447', 'coordX': 67.65400000000002, 'coordY': 53.39099999999999, 'coordZ': -413.139, 'class': 5, 'probability': 0.7757217884063721, 'wtype': 'lung'}
{'seriesuid': '644447', 'coordX': -28.345999999999975, 'coordY': -93.60900000000001, 'coordZ': -372.139, 'class': 5, 'probability': 0.7823397517204285, 'wtype': 'lung'}
{'seriesuid': '644447', 'coordX': 131.65400000000002, 'coordY': -14.609000000000009, 'coordZ': -375.139, 'class': 5, 'probability': 0.9961679577827454, 'wtype': 'lung'}
{'seriesuid': '644447', 'coordX': -35.345999999999975, 'coordY': 16.39099999999999, 'coordZ': -340.139, 'c

20/20 [==============================] - 0s 11ms/sample
{'seriesuid': '688860', 'coordX': 77.0, 'coordY': 151.0, 'coordZ': -466.13, 'class': 1, 'probability': 0.9487388730049133, 'wtype': 'lung'}
{'seriesuid': '688860', 'coordX': 50.0, 'coordY': 129.0, 'coordZ': -425.13, 'class': 5, 'probability': 0.9311882853507996, 'wtype': 'lung'}
{'seriesuid': '688860', 'coordX': 47.0, 'coordY': 188.0, 'coordZ': -426.13, 'class': 5, 'probability': 0.9623960852622986, 'wtype': 'lung'}
{'seriesuid': '688860', 'coordX': 57.0, 'coordY': 153.0, 'coordZ': -514.13, 'class': 5, 'probability': 0.7572588920593262, 'wtype': 'lung'}
{'seriesuid': '688860', 'coordX': 53.0, 'coordY': 166.0, 'coordZ': -511.13, 'class': 5, 'probability': 0.9986214637756348, 'wtype': 'lung'}
{'seriesuid': '688860', 'coordX': -34.0, 'coordY': 132.0, 'coordZ': -491.13, 'class': 5, 'probability': 0.9945419430732727, 'wtype': 'lung'}
{'seriesuid': '688860', 'coordX': -43.0, 'coordY': 197.0, 'coordZ': -493.13, 'class': 5, 'probability':

In [14]:
df_results = pd.read_csv(filepath_or_buffer=settings.SUBMISSION_DIR + 'candidates' + '/submission_' + wtype +'.csv', 
                            index_col=['seriesuid'])
df_results.index = df_results.index.astype('str')

print('results:', df_results.shape, 'distinct lung:', len(set(df_results.index)))

results: (5840, 6) distinct lung: 222


In [15]:
df_results = df_results[df_results['probability'] > 0.95]
df_results = df_results.drop(columns=['wtype'])
print(df_results.shape, 'distinct lung:', len(set(df_results.index)))

(3252, 5) distinct lung: 221


In [16]:
df_results.to_csv(settings.SUBMISSION_DIR + 'candidates' + '/submission_' + wtype + '_filtered.csv', encoding='utf-8')

print('31:', len(df_results[df_results['class']==31]), '32:', len(df_results[df_results['class']==32]))
print('1:', len(df_results[df_results['class']==1]), '5:', len(df_results[df_results['class']==5]))

31: 0 32: 0
1: 709 5: 2543
